<a href="https://colab.research.google.com/github/nhathoang0110/project2/blob/master/RNN_prj2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch  
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import time
import torch.nn.functional as F

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!nvidia-smi

Mon Jun 22 12:41:11 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from time import time


def load_data(data_path):
  with open(data_path, encoding = 'latin1') as f:
    d_lines = f.read().splitlines()
  data, labels, sentence_lengths = [], [], []
  for line in d_lines:
    features = line.split('<fff>')
    label, doc_id, sentence_len = int(features[0]), int(features[1]), int(features[2])
    vector = [int(ID) for ID in features[3].split()]
    data.append(vector)
    labels.append(label)
    sentence_lengths.append(sentence_len)
  return torch.tensor(data), torch.tensor(labels), torch.tensor(sentence_lengths)
  
train_data, train_labels, train_sentence_lengths = load_data(
    data_path='/content/drive/My Drive/20news-train-encoded.txt'
)
test_data, test_labels, test_sentence_lengths = load_data(
    data_path='/content/drive/My Drive/20news-test-encoded.txt'
)
with open('/content/drive/My Drive/vocab-raw.txt', encoding = 'latin1') as f:
  vocab_size = len(f.read().splitlines())


In [31]:
device = torch.device("cuda")
print(device)

cuda


In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import numpy as np

MAX_DOC_LENGTH = 500
NUM_CLASSES = 20
class RNN(nn.Module):
  def __init__(self, vocab_size, embedding_size, lstm_size, batch_size):
    super().__init__()
    self._vocab_size = vocab_size
    self._embedding_size = embedding_size
    self._lstm_size = lstm_size
    self._batch_size = batch_size
    self.build_graph()

  def build_graph(self):
    self._embedding_layer = nn.Embedding(self._vocab_size+2, self._embedding_size)
    self._LSTM_layer = nn.LSTM(self._embedding_size, self._lstm_size, batch_first=True)
    self._full_connected_layer = nn.Linear(self._lstm_size, NUM_CLASSES)
    self._dropout = nn.Dropout(p=0.5)

  def forward(self, data, sentence_lengths):
    h = torch.zeros(1, data.size(0), self._lstm_size).to(device)
    c = torch.zeros(1, data.size(0), self._lstm_size).to(device)
    embeddings = self._embedding_layer(data)  
    lstm_outputs, (hidden, cell)  = self._LSTM_layer(embeddings, (h, c))
    lstm_outputs = [lstm_outputs[i][:sentence_lengths[i]].mean(0) for i in range(data.size(0))]
    lstm_outputs = torch.stack(lstm_outputs)
    outputs = self._dropout(lstm_outputs)
    outputs = self._full_connected_layer(lstm_outputs)
    return outputs


In [45]:
rnn = RNN(
    vocab_size=vocab_size, 
    embedding_size=475, 
    lstm_size=90, 
    batch_size=10,
)
rnn.to(device)

RNN(
  (_embedding_layer): Embedding(18988, 475)
  (_LSTM_layer): LSTM(475, 90, batch_first=True)
  (_full_connected_layer): Linear(in_features=90, out_features=20, bias=True)
  (_dropout): Dropout(p=0.5, inplace=False)
)

In [ ]:
batch_size=10
max_epochs=50
learning_rate=0.01
def test(model, test_data, test_labels, test_sentence_lengths):
  data_set = TensorDataset(test_data, test_labels, test_sentence_lengths)
  data_loader = DataLoader(data_set, batch_size = batch_size, shuffle = False)
  num_true_predict = 0
  for data, labels, sentence_lengths in data_loader:
    data = data.to(device)
    labels = labels.to(device)
    predicted = torch.argmax(model.forward(data, sentence_lengths), dim = 1)
    num_true_predict += sum((predicted == labels).float())
  return num_true_predict*100./len(test_data)

data_set = TensorDataset(train_data, train_labels, train_sentence_lengths)
data_loader = DataLoader(data_set, batch_size = batch_size, shuffle = True)
opt = torch.optim.Adam(rnn.parameters(), lr = learning_rate)
loss_function = nn.CrossEntropyLoss()
rnn.train()
last_loss = 0
for epoch in range(max_epochs):
  new_loss = 0
  for data, labels, sentence_lengths in data_loader:
    data = data.to(device)
    labels = labels.to(device)
    opt.zero_grad()
    prediced = rnn.forward(data, sentence_lengths)
    loss = loss_function(prediced, labels)
    new_loss += loss
    loss.backward()
    opt.step()
  new_loss = new_loss / len(data_loader)
  print('epoch: {}, loss: {}'.format(epoch, new_loss))
  print('test accuracy:',test(rnn,test_data, test_labels, test_sentence_lengths))
  print('train accuracy:', test(rnn,train_data, train_labels, train_sentence_lengths))






epoch: 0, loss: 0.0052651516161859035
test accuracy: tensor(75.4647, device='cuda:0')
train accuracy: tensor(99.9381, device='cuda:0')
epoch: 1, loss: 0.004627746995538473
test accuracy: tensor(75.3186, device='cuda:0')
train accuracy: tensor(99.9470, device='cuda:0')
epoch: 2, loss: 0.002904835855588317
test accuracy: tensor(75.4381, device='cuda:0')
train accuracy: tensor(99.9735, device='cuda:0')
epoch: 3, loss: 0.0019345226464793086
test accuracy: tensor(75.9825, device='cuda:0')
train accuracy: tensor(99.9823, device='cuda:0')
epoch: 4, loss: 0.0018781314138323069
test accuracy: tensor(75.4780, device='cuda:0')
train accuracy: tensor(99.9823, device='cuda:0')
epoch: 5, loss: 0.001613859785720706
test accuracy: tensor(76.0621, device='cuda:0')
train accuracy: tensor(99.9823, device='cuda:0')
epoch: 6, loss: 0.0013219816610217094
test accuracy: tensor(75.7966, device='cuda:0')
train accuracy: tensor(99.9823, device='cuda:0')
epoch: 7, loss: 0.0013298067497089505
test accuracy: tenso